uncomment the below cell to download the latest stack overflow dump and upload to gcs

In [1]:
# %%bash

# sudo apt update && sudo apt install -y p7zip-full

# wget --retry-connrefused --waitretry=1 --read-timeout=20 --timeout=15 -t 0 --continue https://archive.org/download/stackexchange/stackoverflow.com-Posts.7z


# 7z x stackoverflow.com-Posts.7z

# gsutil cp Posts.xml gs://np-training-tmp/stackoverflow/raw/Posts.xml


# wget --retry-connrefused --waitretry=1 --read-timeout=20 --timeout=15 -t 0 --continue https://archive.org/download/stackexchange/stackoverflow.com-PostLinks.7z

# 7z x stackoverflow.com-PostLinks.7z

# gsutil cp PostLinks.xml gs://np-training-tmp/stackoverflow/raw/PostLinks.xml

# ```



In [2]:
!pip install networkx==2.8.7

In [3]:
import pandas as pd
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import networkx as nx

import pyspark.sql


In [4]:
def cleanup_name(name:str):
    return name[1:]

In [5]:
spark = SparkSession\
.builder\
.appName('app')\
.config('spark.jars.packages', 'com.databricks:spark-xml_2.12:0.15.0')\
.config("spark.default.parallelism",200) \
.config("spark.sql.shuffle.partitions",200) \
.getOrCreate()

# .config("spark.sql.crossJoin.enabled",True) \


:: loading settings :: url = jar:file:/usr/lib/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-40d8a075-a991-4789-bd38-b898e6787b86;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.15.0 in central
	found commons-io#commons-io;2.11.0 in central
	found org.glassfish.jaxb#txw2;3.0.2 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.3.0 in central
:: resolution report :: resolve 265ms :: artifacts dl 5ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.15.0 from central in [default]
	commons-io#commons-io;2.11.0 from central in [default]
	org.apache.ws.xmlschema#xmlschema-core;2.3.0 from central in [default]
	org.glassfish.jaxb#txw2;3.0.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlde

In [6]:
input_path = "gs://np-training-tmp/stackoverflow/raw/"
output_path = "gs://np-training-tmp/stackoverflow/output/"

In [7]:
!gsutil ls -lah  {input_path}

985.24 MiB  2022-10-16T23:31:27Z  gs://np-training-tmp/stackoverflow/raw/PostLinks.xml#1665963087716264  metageneration=1
 89.98 GiB  2022-10-16T23:31:27Z  gs://np-training-tmp/stackoverflow/raw/Posts.xml#1665963087770228  metageneration=1
TOTAL: 2 objects, 97652306314 bytes (90.95 GiB)


In [8]:
!gsutil ls -lah  {output_path}

       0 B  2022-10-22T18:49:57Z  gs://np-training-tmp/stackoverflow/output/#1666464597887014  metageneration=1
                                 gs://np-training-tmp/stackoverflow/output/df_links_parquet/
                                 gs://np-training-tmp/stackoverflow/output/df_posts_parquet/
                                 gs://np-training-tmp/stackoverflow/output/posts__links/
                                 gs://np-training-tmp/stackoverflow/output/posts__qn/
                                 gs://np-training-tmp/stackoverflow/output/posts__qn_with_top_answer/
TOTAL: 1 objects, 0 bytes (0 B)


In [ ]:
df_posts_all = spark \
    .read \
    .format("com.databricks.spark.xml") \
    .option('rowTag', 'row') \
    .option('samplingRatio',0.1) \
    .load(input_path+"Posts.xml") \
    .repartition(200)

In [ ]:
df_posts_all = df_posts_all.cache()

In [ ]:
df_posts_all.show(5)

+-----------------+------------+--------------------+-----------+-------------+-------------------+---------------+--------------------+--------------+-------+--------------------+--------------------+----------------------+-----------------+-----------------+------------+---------+-----------+------+--------------------+--------------------+----------+
|_AcceptedAnswerId|_AnswerCount|               _Body|_ClosedDate|_CommentCount|_CommunityOwnedDate|_ContentLicense|       _CreationDate|_FavoriteCount|    _Id|   _LastActivityDate|       _LastEditDate|_LastEditorDisplayName|_LastEditorUserId|_OwnerDisplayName|_OwnerUserId|_ParentId|_PostTypeId|_Score|               _Tags|              _Title|_ViewCount|
+-----------------+------------+--------------------+-----------+-------------+-------------------+---------------+--------------------+--------------+-------+--------------------+--------------------+----------------------+-----------------+-----------------+------------+---------+-----

In [ ]:
df_posts_all.columns

['_AcceptedAnswerId',
 '_AnswerCount',
 '_Body',
 '_ClosedDate',
 '_CommentCount',
 '_CommunityOwnedDate',
 '_ContentLicense',
 '_CreationDate',
 '_FavoriteCount',
 '_Id',
 '_LastActivityDate',
 '_LastEditDate',
 '_LastEditorDisplayName',
 '_LastEditorUserId',
 '_OwnerDisplayName',
 '_OwnerUserId',
 '_ParentId',
 '_PostTypeId',
 '_Score',
 '_Tags',
 '_Title',
 '_ViewCount']

In [ ]:
column_names = [cleanup_name(name) for name in df_posts_all.columns ] 

In [ ]:
len(column_names)

22

In [ ]:
df_posts_all = df_posts_all.toDF(*column_names)


In [ ]:
df_posts_all.show(5)

+----------------+-----------+--------------------+----------+------------+------------------+--------------+--------------------+-------------+-------+--------------------+--------------------+---------------------+----------------+----------------+-----------+--------+----------+-----+--------------------+--------------------+---------+
|AcceptedAnswerId|AnswerCount|                Body|ClosedDate|CommentCount|CommunityOwnedDate|ContentLicense|        CreationDate|FavoriteCount|     Id|    LastActivityDate|        LastEditDate|LastEditorDisplayName|LastEditorUserId|OwnerDisplayName|OwnerUserId|ParentId|PostTypeId|Score|                Tags|               Title|ViewCount|
+----------------+-----------+--------------------+----------+------------+------------------+--------------+--------------------+-------------+-------+--------------------+--------------------+---------------------+----------------+----------------+-----------+--------+----------+-----+--------------------+---------

In [ ]:
df_posts_all.repartition(200).write.mode("overwrite").parquet(f"{output_path}/df_posts_parquet")

In [ ]:
df_posts_all = spark.read.parquet(f"{output_path}/df_posts_parquet")

In [ ]:
df_posts_all.count()

56264788

Questions

In [ ]:
df_qns = df_posts_all\
    .where(F.col("PostTypeId")==1) \
    .selectExpr("Id","AcceptedAnswerId","Title","Body as QuestionBody","Tags", "ViewCount", "AnswerCount",  "CommentCount" , "Score", "CreationDate") \
    .persist()

In [ ]:
df_qns.show(5)

+--------+----------------+--------------------+--------------------+--------------------+---------+-----------+------------+-----+--------------------+
|      Id|AcceptedAnswerId|               Title|        QuestionBody|                Tags|ViewCount|AnswerCount|CommentCount|Score|        CreationDate|
+--------+----------------+--------------------+--------------------+--------------------+---------+-----------+------------+-----+--------------------+
|45735822|        45736088|Scala java.lang.N...|<p>I am trying to...|<postgresql><scal...|     1367|          1|           0|    1|2017-08-17 13:00:...|
|65918510|            null|Convert a HTML th...|<p>I have an HTML...|<python><html><be...|       36|          1|           0|    1|2021-01-27 11:47:...|
|56841245|        56841350|Dates array using...|<p>Our team is ju...|<javascript><arra...|     5425|          5|           2|    1|2019-07-01 19:15:...|
|26375136|            null|Openshift - conec...|<p>I did the rhc ...|<postgresql><

In [ ]:
df_qns.count()

22634239

In [ ]:
df_qns.repartition(30).write.mode("overwrite").parquet(f"{output_path}/posts__qn")

Answers

In [ ]:
df_answers = df_posts_all\
    .where(F.col("PostTypeId")==2) \
    .selectExpr("Id as AnswerId","Body as AnswerBody") \
    .persist()


In [ ]:
df_answers.show(5)

+--------+--------------------+
|AnswerId|          AnswerBody|
+--------+--------------------+
|24802250|<p>I'm pretty sur...|
|22825188|<p>When you are p...|
|51626960|<p>React v15.5.x ...|
|14828282|<p>You have sever...|
|42739536|<p>According to :...|
+--------+--------------------+
only showing top 5 rows



In [ ]:
df_answers.count()

33520483

Create Post + Top Answer

In [ ]:
# df_posts_final = df_qns.join(df_answers
#     , on=[ 
#         df_qns.AcceptedAnswerId == df_answers.Id
#     ]
#     , how="inner"
#     )

In [ ]:
df_posts_final = df_qns.alias("qn").join(df_answers.alias("answer")
    , on=[ 
        F.col("qn.AcceptedAnswerId") == F.col("answer.AnswerId") 
    ]
    , how="left"
    ) \
    .withColumnRenamed("AnswerBody", "AcceptedAnswerBody")
#     .drop(F.col("answer.AnswerId")) \


In [ ]:
df_posts_final.show(5)

+-----+----------------+--------------------+--------------------+--------------------+---------+-----------+------------+-----+--------------------+--------+--------------------+
|   Id|AcceptedAnswerId|               Title|        QuestionBody|                Tags|ViewCount|AnswerCount|CommentCount|Score|        CreationDate|AnswerId|  AcceptedAnswerBody|
+-----+----------------+--------------------+--------------------+--------------------+---------+-----------+------------+-----+--------------------+--------+--------------------+
|   17|              26|Binary Data in MySQL|<p>How do I store...|<mysql><database>...|    83953|          9|           3|  194|2008-08-01 05:09:...|      26|<p>The answer by ...|
| 2873|            2927|Choosing a static...|<p>I'm working on...|  <c><unix><testing>|    55970|         15|           3|   69|2008-08-05 21:19:...|    2927|<p>Don't overlook...|
| 7224|            7225|Change the width ...|<p>Is it possible...| <vb.net><scrollbar>|    16007|   

In [ ]:
df_posts_final.count()

22634239

In [ ]:
df_posts_final\
.where(F.col("AnswerId").isNotNull()) \
.count()

11588225

In [ ]:
df_posts_final.repartition(30).write.mode("overwrite").parquet(f"{output_path}/posts__qn_with_top_answer")

## Links

In [ ]:
df_links = spark \
    .read \
    .format("com.databricks.spark.xml") \
    .option('rowTag', 'row') \
    .option('samplingRatio',0.1) \
    .load(input_path+"PostLinks.xml")

In [ ]:
df_links.columns

['_CreationDate', '_Id', '_LinkTypeId', '_PostId', '_RelatedPostId']

In [ ]:
column_names = [cleanup_name(name) for name in df_links.columns ] 

In [ ]:
df_links = df_links.toDF(*column_names)


In [ ]:
df_links = df_links \
    .where(F.col("LinkTypeId")==3) \
    .select("PostId","RelatedPostId") \
    .persist()

In [ ]:
df_links.count()

1347017

In [ ]:
df_links.repartition(30).write.mode("overwrite").parquet(f"{output_path}/df_links_parquet")

In [ ]:
df_links = spark.read.parquet(f"{output_path}/df_links_parquet")

In [ ]:
G = nx.Graph()

In [ ]:
for record in df_links.toPandas().itertuples():
    #print (record.PostId, record.RelatedPostId)
    G.add_edge(record.PostId, record.RelatedPostId)

In [ ]:
nx.number_connected_components(G)

371276

In [ ]:
type(G)

networkx.classes.graph.Graph

In [ ]:
def get_connected_component(G):
    components = [G.subgraph(c).copy() for c in nx.connected_components(G)]

    records = [] 
    for idx,g in enumerate(components,start=1):
        #print(f"Component {idx}: Nodes: {g.nodes()} Edges: {g.edges()}")

        records.append ( {
            "component_id": idx
            ,"nodes": list( g.nodes )
        })
    df = pd.DataFrame(records)

    return df

In [ ]:
pdf_connected_component = get_connected_component(G)
pdf_connected_component = pdf_connected_component.explode("nodes").rename(columns={'nodes':'Id'})

In [ ]:
pdf_connected_component

,component_id,Id
0,1,64927065
0,1,47138754
1,2,2102746
1,2,59383746
1,2,64921629
...,...,...
371273,371274,32526836
371274,371275,53434490
371274,371275,52309940
371275,371276,42031514


In [ ]:
df_connected_component = spark.createDataFrame(pdf_connected_component) 


In [ ]:
df_connected_component.show(5)

+------------+--------+
|component_id|      Id|
+------------+--------+
|           1|64927065|
|           1|47138754|
|           2| 2102746|
|           2|59383746|
|           2|64921629|
+------------+--------+
only showing top 5 rows



In [ ]:
df_connected_component.count()

1652946

In [ ]:
# from pyspark.sql.window import Window
# from pyspark.sql.functions import col, row_number


w2 = pyspark.sql.window.Window.partitionBy("component_id").orderBy(F.col("ViewCount").desc())




df_top_question_in_connected_component = df_connected_component\
    .join(df_qns,on=["Id"]) \
    .select("component_id","Id","Title","QuestionBody","ViewCount","Score") \
    .withColumn("row",F.row_number().over(w2)) \
    .filter(F.col("row") == 1).drop("row") \
    .selectExpr("component_id", "Id as OtherPostId")

In [ ]:
df_top_question_in_connected_component.show()

+------------+-----------+
|component_id|OtherPostId|
+------------+-----------+
|          26|   19277973|
|          29|   31126124|
|         474|   49612412|
|         964|   11290323|
|        1677|   34338131|
|        1697|   33729066|
|        1806|     203844|
|        1950|    1560424|
|        2040|    9913796|
|        2214|   63661111|
|        2250|   10134310|
|        2453|    2757549|
|        2509|   53758761|
|        2529|   60651615|
|        2927|    2440511|
|        3091|    4822953|
|        3506|   12115357|
|        3764|   50772328|
|        4590|   32443402|
|        4823|    2751593|
+------------+-----------+
only showing top 20 rows



In [ ]:
df_link_top = df_connected_component\
    .join(df_top_question_in_connected_component, on=["component_id"]) \
    .where (F.col("Id") != F.col("OtherPostId"))

In [ ]:
df_link_top.show()

+------------+--------+-----------+
|component_id|      Id|OtherPostId|
+------------+--------+-----------+
|          26|66528713|   19277973|
|          29|48868371|   31126124|
|          29|48865068|   31126124|
|          29|34256132|   31126124|
|          29|67412386|   31126124|
|         474|66221740|   49612412|
|         964| 2065495|   11290323|
|         964|68438511|   11290323|
|        1677|41892662|   34338131|
|        1677|64139523|   34338131|
|        1697|65435265|   33729066|
|        1806| 5798431|     203844|
|        1806|34882750|     203844|
|        1806|37013783|     203844|
|        1806|27452694|     203844|
|        1806|55530037|     203844|
|        1806| 1629906|     203844|
|        1806|37423790|     203844|
|        1806|10666890|     203844|
|        1806|62972874|     203844|
+------------+--------+-----------+
only showing top 20 rows



In [ ]:
df_link_top.count()

1266922

In [ ]:
df_link_top.dropDuplicates(['component_id']).count()

358303

In [62]:
df_link_top.repartition(30).write.mode("overwrite").parquet(f"{output_path}/posts__links")

In [63]:
!ls

bin	   dev	   home   lib64       media  proc  sbin  tmp
boot	   etc	   lib	  libx32      mnt    root  srv	 usr
copyright  hadoop  lib32  lost+found  opt    run   sys	 var


In [64]:
df_tmp = spark.read.parquet(f"{output_path}/posts__links")

In [65]:
df_tmp.show(5)

+------------+--------+-----------+
|component_id|      Id|OtherPostId|
+------------+--------+-----------+
|       11857|48483506|   40872114|
|      111173|26220432|   13252603|
|       41872|24063986|      32332|
|      248533|31704180|    1803627|
|       49609|53596207|   53524127|
+------------+--------+-----------+
only showing top 5 rows



----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 58850)
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/miniconda3/lib/python3.8/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/miniconda3/lib/python3.8/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/miniconda3/lib/python3.8/socketserver.py", line 747, in __init__
    self.handle()
  File "/usr/lib/spark/python/pyspark/accumulators.py", line 262, in handle
    poll(accum_updates)
  File "/usr/lib/spark/python/pyspark/accumulators.py", line 235, in poll
    if func():
  File "/usr/lib/spark/python/pyspark/accumulators.py", line 239, in accum_updates
    num_updates = read_int(self.rfile